# Project objectives:
1. Understand what is  A* search algorithm is and how it works
1. Real Example of A* algorithm.
1. We will explain the difference between Manhattan, Euclidean, and Chebyshev distance.

# Introduction:
In this project, we will explain the A* search algorithm (Informed search algorithms) and how it depends on the principle of (Dijkstra) shortest path to give us a better and faster solution. Also, we will explain the difference between Manhattan, Euclidean, and Chebyshev distance when we are dealing with problems that are related to finding the shortest path between two things. It does this by introducing a heuristic element to decide the node that is supposed to be followed.

# Route finding problem:
we focused on finding a way to exit a maze by getting the shortest path by implementing the A* search algorithm and testing several heuristic functions to find the best one.

## Problem formulation:
- Initial state: a selected square (start square)
- Actions: North, South, East, West, North-East, South-East, South-West, North-West
- State-space: 30X60 squares
- Goal state: Exit the maze by going to the selected square(goal square).

![image info](./images/Maze.png)


# What is the A* search algorithm?
A* is an informed search algorithm, sometimes known as a best-first search because it is written in terms of weighted graphs. Its goal is to discover the shortest path from a given starting node to a particular goal node (least distance traveled, shortest time, etc.). It accomplishes this by keeping track of a tree of pathways that begin at the start node and extending those paths one edge at a time until the termination requirement is met.

# Heuristic function?
is a function in search algorithms that rates alternatives depending on available information at each branching step to determine which branch to take. There is 5 behavior for heuristic:
1. If the heuristic is 0 then the actual cost g(n) will only be used, and A* will be turned into Dijkstra’s algorithms
1. If the heuristic is lower than the actual path, then A* will guarantee the shortest path,  and the more heuristic being lower the more expanded node happens.
1. If the heuristic is the same as an actual path, then A* will find the shortest path with exact step without expanding any unnecessary node
1. If the heuristic is greater than the actual path, the A* may not guarantee the shortest path, but it will run faster
1. If the heuristic is greater by an extreme factor, the A* will turn into Gready algorithms

We will test most of these in this tutorial.

## Euclidean distance:
Euclidean distance is The shortest path between source and destination or between a point and another, which is usually described by a straight line. It is useful for things like locating the closest hospital for an emergency helicopter flight. When constructing a suitability map, this tool may also be used to obtain data representing the distance from a certain item.	

## Manhattan distance:
If we need to determine the distance between two data points on a grid-like route, we utilize Manhattan distance. For Example, if we want to calculate the distance from a place to another and we are moved by a vehicle then we will consider that we will face a lot of obstacles or walls so in that case, we will need to turn in another direction to continue moving. So, we need to use Manhattan distance.


## Chebyshev distance:
The biggest difference between two vectors along any coordinate dimension is known as the Chebyshev distance. To put it another way, it's the greatest distance along one axis. Because the smallest number of moves required by a king to move from one square to another is equal to Chebyshev distance, it is sometimes referred to as Chessboard distance. 

![image info](./images/heuristic.png)


# A* Algorithm:
Like Dijkstra, A* works by making a most minimal expense way tree from the beginning hub to the objective hub. What makes A* unique and better for some hunts is that for every hub, A* utilizes a capacity f(n) that gives a gauge of the absolute expense of a way utilizing that hub. Consequently, A* is a heuristic capacity, which contrasts from a calculation in that a heuristic is a greater amount of a gauge and isn't really provably right.

We will start implementing the minimum environment for A* to work, then we will take what we learn and change it to interactive and visualize GUI to increase our understanding of the algorithm.
## Preparation:
Before starting to implement A*, we need an environment for A* to work on. To recap, what we will implement is a maze in the shape of a grid, each cell in the grid will have a value that will be stored in a two-dimensional array, the maze will represent as follow:

![image info](./images/Grid.png)

It starts from the top left as (0, 0) to the bottom-right (9, 12). Each cell will have the ability to either go in 4 directions or 8 directions. The direction will be controlled by the user.

![image info](./images/Direction.png)

## Implementation:
We will start implementing the minimum environment for A* to work, then we will take what we learn and change it to interactive and visualize GUI to increase our understanding of the algorithm.

We will start with Location and Cell type:

In [3]:
from enum import Enum
from typing import NamedTuple


class Cell(str, Enum):
    BLOCK = "#"
    START = "S"
    GOAL = "G"
    PATH = "*"
    EMPTY = " "

class Location(NamedTuple):
    row: int
    column: int

In [4]:
import random
from typing import List


class Maze:
    def __init__(self, rows: int, columns: int, start_location: Location, goal_location: Location):
        self._rows = rows
        self._columns = columns
        self._start: Location = start_location
        self._goal: Location = goal_location
        self._grid = [[Cell.EMPTY for _ in range(columns)]
                      for _ in range(rows)]
        self._grid[start_location.row][start_location.column] = Cell.START
        self._grid[goal_location.row][goal_location.column] = Cell.GOAL
        self._fill_random()

    def successor(self, location: Location, allow_diagonal=False) -> List[Location]:
        """
        Given a location, return all the possible state of the location
        :param location
        :param allow_diagonal: default False
        :return: list of available location
        """
        list_of_neighbor: List[Location] = []
        if location.column - 1 >= 0 and self._grid[location.row][location.column - 1] != Cell.BLOCK:
            list_of_neighbor.append(Location(location.row, location.column - 1))
        if location.row + 1 < self._rows and self._grid[location.row + 1][location.column] != Cell.BLOCK:
            list_of_neighbor.append(Location(location.row + 1, location.column))
        if location.column + 1 < self._columns and self._grid[location.row][location.column + 1] != Cell.BLOCK:
            list_of_neighbor.append(Location(location.row, location.column + 1))
        if location.row - 1 >= 0 and self._grid[location.row - 1][location.column] != Cell.BLOCK:
            list_of_neighbor.append(Location(location.row - 1, location.column))
        if not allow_diagonal:
            return list_of_neighbor
        # top-right
        if (location.column - 1 >= 0 and location.row + 1 < self._rows) and self._grid[
            location.row + 1][location.column - 1] != Cell.BLOCK:
            list_of_neighbor.append(Location(location.row + 1, location.column - 1))
        # bottom-right
        if (location.column + 1 < self._columns and location.row + 1 < self._rows) and self._grid[
            location.row + 1][location.column + 1] != Cell.BLOCK:
            list_of_neighbor.append(Location(location.row + 1, location.column + 1))
        # bottom - left
        if (location.column + 1 < self._columns and location.row - 1 >= 0) and self._grid[
            location.row - 1][location.column + 1] != Cell.BLOCK:
            list_of_neighbor.append(Location(location.row - 1, location.column + 1))
        # top-left
        if (location.column - 1 >= 0 and location.row - 1 >= 0) and self._grid[
            location.row - 1][location.column - 1] != Cell.BLOCK:
            list_of_neighbor.append(Location(location.row - 1, location.column - 1))
        return list_of_neighbor

    def goal_check(self, location: Location) -> bool:
        """
        Check if the location is the same as the goal
        :param location
        :return: bool
        """
        return self._goal == location

    def mark(self, paths: List[Location]):
        """
        Mark all the location in the path as PATH
        :param paths: List of location
        """
        for location in paths:
            self._grid[location.row][location.column] = Cell.PATH
        self._grid[self._start.row][self._start.column] = Cell.START
        self._grid[self._goal.row][self._goal.column] = Cell.GOAL

    def __str__(self):
        """
        represent maze in the terminal
        :return: maze representation: str
        """
        output = ""
        for row in self._grid:
            output += "".join([cell.value for cell in row]) + "\n"
        return output

    def _fill_random(self):
        """
        Create random block, each cell have 0.2 chance to be BLOCK 
        """
        for i, row in enumerate(self._grid):
            for j, value in enumerate(row):
                if value == Cell.EMPTY and random.uniform(0, 1) < 0.2:
                    self._grid[i][j] = Cell.BLOCK

For A* to work in a maze it needs a Node that holds the location, where it comes from, the cost to this location, and heuristic value to the goal. The priority queue will be used as a list to hold Node.

In [20]:
from heapq import heappop, heappush

class Node:
    def __init__(self, location: Location, parent, cost: int, heuristic: float):
        """
        Initialize for Node 
        :param location: where the node located in the maze
        :param parent: the previous location
        :param cost: the actual cost from the start to this location
        :param heuristic: the heuristic value to reach the goal
        """
        self.location = location
        self.parent = parent
        self.cost = cost
        self.heuristic = heuristic

    def __lt__(self, other):
        """
        This method will be used by the priority queue to determine the lower value
        :param other
        :return: bool
        """
        return (self.cost + self.heuristic) < (other.cost + other.heuristic)
    

class PriorityQueue:
    """
    We will use the built-in implementation for adding Node and pulling Node from the list
    """
    def __init__(self):
        """
        Initialize the list
        """
        self._list = []

    @property
    def empty(self):
        """Checking property if the list is empty"""
        return not self._list



    def push(self, node: Node):
        """
        Adding element using heappush method from python
        :param node: Node
        """
        heappush(self._list, node)

    def pop(self) -> Node:
        """
        Return the lower value in the list using the heappop from python
        :return: Node
        """
        return heappop(self._list)

After this we need the heuristic functions and a function to get the path from a node to the start node(traceback):

In [7]:
from typing import Callable

def euclidean_distance(goal: Location) -> Callable[[Location], float]:
    def distance(location: Location):
        return sqrt((goal.row - location.row) ** 2 + (goal.column - location.column) ** 2)

    return distance


def manhattan_distance(goal: Location) -> Callable[[Location], float]:
    def distance(location: Location):
        return abs(goal.row - location.row) + abs(goal.column - location.column)

    return distance

def chebyshev_distance(goal: Location) -> Callable[[Location], float]:
    def distance(location: Location):
        """
        D * (dx + dy) + (D2 - 2 * D) * min(dx, dy)
        D and D2 the cost of going diagonally are cost, which in our case is 1
        :param location:
        :return: float
        """
        dx = abs(goal.row - location.row)
        dy = abs(goal.column - location.column)
        return (dx + dy) + (-1) * min(dx, dy)

    return distance

def path_to_node(node: Node):
    """Return the list from the giving node to the start node"""
    path = []
    while node is not None:
        path.append(node.location)
        node = node.parent
    return path[::-1]

## A*:
Now we can start to implement A* after all these preparation. A* is a straightforward and simple algorithm, to work it need:
- Starting location: the starting location of the maze.
- Goal testing: function to test is the current location is the goal.
- Successor: a function that takes location and returns all the possible location.
- Heuristic: a function that estimates the distance between location and goal.
We will store all nodes that need to be explored in the priority queue to increase the performance and will use a dictionary for location and cost associated with it.


In [8]:
def a_star(start: Location, goal_check: Callable[[Location], bool],
          successor: Callable[[Location, bool], List[Location]], heuristic: Callable[[Location], float],
          allow_diagonal: bool):
   """
   Implementation of A* algorithms
   :param start: start location
   :param goal_check: reference of the method to test if the location is the goal
   :param successor: reference of the method to get all the available locations from giving the location
   :param heuristic: reference of the method to get the heuristic cost of giving location to goal location
   :param allow_diagonal: allow for diagonal movement
   :return:
   """
   frontier = PriorityQueue()
   node = Node(start, None, 0, heuristic(start))
   frontier.push(node)
   explored: Dict[Location, int] = {start: 0}
   while not frontier.empty:
       current_node = frontier.pop()
       if goal_check(current_node.location):
           return current_node
       for child in successor(current_node.location, allow_diagonal=allow_diagonal):
           new_cost = current_node.cost + 1
           if child not in explored or explored[child] > new_cost:
               frontier.push(Node(child, current_node, new_cost, heuristic(child)))
               explored[child] = new_cost
   return None # No solution found!

## Testing:
After implementing A*, we can start testing all things together.

In [13]:
start = Location(0, 0)
goal = Location(29, 59)
euclidean_heuristic = euclidean_distance(goal)
manhattan_heuristic = manhattan_distance(goal)
chebyshev_heuristic = chebyshev_distance(goal)
maze: Maze = Maze(30, 60, start, goal)
solution: Node = a_star(start, maze.goal_check, maze.successor, manhattan_heuristic, False)
print(maze)
if not solution:
   print("No solution is found")
else:
   path = path_to_node(solution)
   maze.mark(path)
   print(maze)

S  # #                  #         # # ##        ##    #     
#   #   #   #         #    ##    #   #  ##   #      ## #####
 #     #     # #   ####          # #  # #          #   #    
     #     #   # # #    #  #  # ###    #      # #      #### 
#            #### #     #  # #                 # #######   #
 #   #              #             #   #            # #  #   
         #   #   ###  #  #   ### #  ### #   #      # #      
 #            ##             #  #       #    #  ###         
 #     ####   #  #                #  #  # ##   ####     #   
    # ##       #       ##             #   ##             # #
 #   # # #             #                   #  ##        #   
# #      #   #    #         ##     #    #    ##    ##    # #
          ## ## #                     #      #  #   #       
   #    ## #       #        ##  # #       #     ##  ##      
   # #            #  # ## #  ##   #  #       ##    #   #   #
         #       ###   ##        #       #   #      #       
#       #  #    #     # 

# Part 2: Implementing Interactive A*:
We will implement A* using pygame library, and before starting with the code, we need some architecture to help us implement the code.
We will have 4 classes each class will have :
- Maze: will be responsible for handling user input and placing it in the maze 
- MazeController: will be responsible for handling user input and making the change in the maze
- AStar: will be responsible for searching 
- SearchController: will be responsible for handling configuration of AStar

We will also modify the Cell class and change it from string to tuple that contains RGB values.


In [15]:
from typing import Tuple

class Cell(Tuple, Enum):
    BLOCK = (41, 41, 41)
    WHITE = (245, 245, 245)
    GARY = (220, 220, 220)
    PATH = (119, 214, 140)
    EXPLORE = (121, 119, 214)
    GOAL = (204, 60, 100)
    START = (255, 153, 0)

In [16]:
!pip install pygame

You should consider upgrading via the '/Users/a7/Documents/Study/university/Level5/Artificial Intelligence/projects/a_star_pygame/venv/bin/python -m pip install --upgrade pip' command.


In [17]:
import pygame


class Maze:
    """
    Representing Maze in pygame
    """
    def __init__(self, rows: int, columns: int,
                 grid_size: int, x_offset: int, y_offset: int,
                 display_surface: pygame.Surface,
                 start: Location = None, goal: Location = None):
        self.x = columns
        self.y = rows
        self._grid_size: int = grid_size
        self._rows: int = self.x // self._grid_size
        self._columns: int = self.y // self._grid_size
        self._x_offset: int = x_offset
        self._y_offset: int = y_offset
        self._start: Location = start
        self._goal: Location = goal
        self.display_surface = display_surface

        self._grid = [[Cell.WHITE for _ in range(self._columns)]
                      for _ in range(self._rows)]

    def check_goal(self, location: Location) -> bool:
        return self._goal == location

    def successor(self, location: Location, allow_diagonal=False) -> List[Location]:
        list_of_neighbor: List[Location] = []
        # the order: Top, Right, Bottom, Left
        if location.column - 1 >= 0 and self._grid[location.row][location.column - 1] != Cell.BLOCK:
            list_of_neighbor.append(Location(location.row, location.column - 1))
        if location.row + 1 < self._rows and self._grid[location.row + 1][location.column] != Cell.BLOCK:
            list_of_neighbor.append(Location(location.row + 1, location.column))
        if location.column + 1 < self._columns and self._grid[location.row][location.column + 1] != Cell.BLOCK:
            list_of_neighbor.append(Location(location.row, location.column + 1))
        if location.row - 1 >= 0 and self._grid[location.row - 1][location.column] != Cell.BLOCK:
            list_of_neighbor.append(Location(location.row - 1, location.column))
        # Check for diagonal neighbor if possible
        if not allow_diagonal:
            return list_of_neighbor
        # top-right
        if (location.column - 1 >= 0 and location.row + 1 < self._rows) and self._grid[
            location.row + 1][location.column - 1] != Cell.BLOCK:
            list_of_neighbor.append(Location(location.row + 1, location.column - 1))
        # bottom-right
        if (location.column + 1 < self._columns and location.row + 1 < self._rows) and self._grid[
            location.row + 1][location.column + 1] != Cell.BLOCK:
            list_of_neighbor.append(Location(location.row + 1, location.column + 1))
        # bottom - left
        if (location.column + 1 < self._columns and location.row - 1 >= 0) and self._grid[
            location.row - 1][location.column + 1] != Cell.BLOCK:
            list_of_neighbor.append(Location(location.row - 1, location.column + 1))
        # top-left
        if (location.column - 1 >= 0 and location.row - 1 >= 0) and self._grid[
            location.row - 1][location.column - 1] != Cell.BLOCK:
            list_of_neighbor.append(Location(location.row - 1, location.column - 1))

        return list_of_neighbor

    def mark(self, location: Location, color: Cell) -> None:
        value = self._grid[location.row][location.column]
        if value != Cell.START and value != Cell.GOAL:
            self._grid[location.row][location.column] = color

    def draw(self):
        # vertical line
        self._draw_vertical_line()
        # horizontal line
        self._draw_horizontal_line()

        self._draw_occupied_place()

    @property
    def start(self) -> Location:
        return self._start

    @start.setter
    def start(self, location: Location):
        start: Location = self._normalize_location(location=location) # Location will
        # Check the boundary before assign
        if not self._check_boundary(location=start):
            return None
        self._start = start
        # check if there are start point in grid before adding the new one
        self._change_location_state(from_state=Cell.START, to=Cell.WHITE)
        self._grid[self._start.row][self._start.column] = Cell.START

    @property
    def goal(self) -> Location:
        return self._goal

    @goal.setter
    def goal(self, location: Location):
        goal: Location = self._normalize_location(location=location)
        # check for boundary before assign
        if not self._check_boundary(location=goal):
            return None
        self._goal = goal
        self._change_location_state(from_state=Cell.GOAL, to=Cell.WHITE)
        self._grid[self._goal.row][self._goal.column] = Cell.GOAL

    def block(self, location: Location, to: Cell = Cell.BLOCK):
        block: Location = self._normalize_location(location=location)
        # check for boundary before assign
        if not self._check_boundary(location=block):
            return None
        if self._grid[block.row][block.column] == Cell.WHITE or self._grid[block.row][block.column] == Cell.BLOCK:
            self._grid[block.row][block.column] = to

    def erase_maze(self):
        self._grid = [[Cell.WHITE for _ in range(self._columns)]
                      for _ in range(self._rows)]

    def clear_maze(self):
        for i, row in enumerate(self._grid):
            for j, value in enumerate(row):
                if value == Cell.EXPLORE or value == Cell.PATH:
                    self._grid[i][j] = Cell.WHITE

    def fill_random(self, spread: float = 0.2):
        for i, row in enumerate(self._grid):
            for j, value in enumerate(row):
                if value == Cell.WHITE and random.uniform(0, 1) < spread:
                    self._grid[i][j] = Cell.BLOCK
                    self.draw()
                    pygame.display.update()

    def _draw_horizontal_line(self):
        # Draw Horizontal Line
        for i in range(self._columns + 1):
            pygame.draw.line(self.display_surface, Cell.GARY,
                             (self._x_offset, i * self._grid_size + self._y_offset),
                             (self._x_offset + self.x, (i * self._grid_size) + self._y_offset))

    def _draw_vertical_line(self):
        # Draw Vertical Line
        for i in range(self._rows + 1):
            pygame.draw.line(self.display_surface, Cell.GARY,
                             (i * self._grid_size + self._x_offset, self._y_offset),
                             (i * self._grid_size + self._x_offset, self._y_offset + self.y))

    def _change_location_state(self, from_state: Cell, to: Cell):
        for i, row in enumerate(self._grid):
            for j, value in enumerate(row):
                if value == from_state:
                    self._grid[i][j] = to

    def _normalize_location(self, location: Location) -> Location:
        x = (location.row - self._x_offset) // self._grid_size
        y = (location.column - self._y_offset) // self._grid_size
        return Location(x, y)

    def _draw_occupied_place(self):
        for i, row in enumerate(self._grid):
            for j, value in enumerate(row):
                if value is not Cell.WHITE:
                    self._draw_rect(x=i, y=j, value=value)

    def _draw_rect(self, x: int, y: int, value: Cell):
        rect_x = (x * self._grid_size) + self._x_offset
        rect_y = (y * self._grid_size) + self._y_offset
        pygame.draw.rect(self.display_surface, value,
                         (rect_x + 1, rect_y + 1, self._grid_size - 1, self._grid_size - 1))

    def _check_boundary(self, location: Location):
        if 0 <= location.row < self._rows and 0 <= location.column < self._columns:
            return True
        return False


pygame 2.1.0 (SDL 2.0.16, Python 3.9.4)
Hello from the pygame community. https://www.pygame.org/contribute.html


The MazeController will take input from the user via keyboard and mouse to execute the following commands:
- Block: mark the current mouse location as Block
- Start: mark the current mouse location as Start
- Goal: mark the current mouse location as Goal
- Erase: erase everything from the maze
- Clear: clear the path from the maze
- Fill: fill the maze with random blocks


In [18]:
class MazeControllerCommand(str, Enum):
    BLOCK = "BLOCK"
    START = "START"
    GOAL = "GOAL"
    ERASE = "ERASE"
    CLEAR = "ERASE EXPLORE AND PATH"
    FILL = "FILL"


class MazeController:
    def __init__(self, maze: Maze, display_surface: pygame.Surface):
        self._maze = maze
        self.display_grid = display_surface
        self._current_command: MazeControllerCommand = MazeControllerCommand.START

    def update(self, events: pygame.event):
        # Loop for event and allow user to:
        for event in events:
            if event.type == pygame.KEYDOWN:
                self._change_current_command(event)
            if self._current_command == MazeControllerCommand.START:
                self._check_for_start_point(event=event)
            if self._current_command == MazeControllerCommand.GOAL:
                self._check_for_goal_point(event=event)
            if self._current_command == MazeControllerCommand.BLOCK:
                self._check_for_block_point(event=event)
            if self._current_command == MazeControllerCommand.ERASE:
                self._erase_maze()
                self._current_command = MazeControllerCommand.START
            if self._current_command == MazeControllerCommand.CLEAR:
                self._clear_path()
            if self._current_command == MazeControllerCommand.FILL:
                self._fill_maze()

    def _change_current_command(self, event):
        if event.key == pygame.K_s:
            self._current_command = MazeControllerCommand.START
        if event.key == pygame.K_g:
            self._current_command = MazeControllerCommand.GOAL
        if event.key == pygame.K_b:
            self._current_command = MazeControllerCommand.BLOCK
        if event.key == pygame.K_e:
            self._current_command = MazeControllerCommand.ERASE
        if event.key == pygame.K_c:
            self._current_command = MazeControllerCommand.CLEAR
        if event.key == pygame.K_f:
            self._current_command = MazeControllerCommand.FILL

    def _check_for_start_point(self, event: pygame.event):
        # press 's' and mouse click, then add start point to maze
        if event.type == pygame.MOUSEBUTTONUP:
            self._maze.start = (Location(row=event.pos[0], column=event.pos[1]))

    def _check_for_goal_point(self, event: pygame.event):
        # press 'g' and mouse click, then add goal point to maze
        if event.type == pygame.MOUSEBUTTONUP:
            self._maze.goal = (Location(row=event.pos[0], column=event.pos[1]))

    def _check_for_block_point(self, event: pygame.event):
        if event.type == pygame.MOUSEMOTION and event.buttons[0]:
            self._maze.block(location=Location(event.pos[0], column=event.pos[1]))
        if event.type == pygame.MOUSEBUTTONUP:
            self._maze.block(location=Location(row=event.pos[0], column=event.pos[1]), to=Cell.WHITE)

    def _erase_maze(self):
        self._maze.erase_maze()

    def _clear_path(self):
        self._maze.clear_maze()
        self._current_command = MazeControllerCommand.BLOCK

    def _fill_maze(self):
        self._maze.fill_random()
        self._current_command = MazeControllerCommand.START


AStar class will be the same as above plus adding additional functionality for measuring time, and all the parameters of Astar will be in the initialize function:

In [21]:
import time
from typing import Dict, List, Union, Callable

import pygame

from maze import Maze
from utils import (PriorityQueue, Node,
                   path_to_node,
                   Location, Cell)


class AStar:
    def __init__(self, heuristic: Callable[[Location], float],
                 successor: Callable[[Location, bool], List[Location]],
                 goal_check: Callable[[Location], bool],
                 start: Location, maze: Maze,
                 allow_diagonal: bool):
        self._heuristic: Callable[[Location], float] = heuristic
        self._successor = successor
        self._goal_check = goal_check
        self._start = start
        self._maze: Maze = maze
        self._solution: Union[Node, None] = None
        self._time: float = 0.0
        self._len_of_path: int = 0
        self._allow_diagonal = allow_diagonal

    @property
    def heuristic(self):
        return self._heuristic

    @heuristic.setter
    def heuristic(self, heuristic):
        self._heuristic = heuristic

    def start(self):
        t1 = time.perf_counter()
        self._solution: Node = self._search()
        t2 = time.perf_counter()
        self._time = t2 - t1
        print(f"time to take: {t2 - t1}")

        if not self._solution:
            print("No solution found")
            return
        paths: List[Location] = path_to_node(self._solution)
        self._len_of_path = len(paths)
        for location in paths:
            self._maze.mark(location=location, color=Cell.PATH)
            self._maze.draw()
            pygame.display.update()

    def _search(self) -> Node:
        # A*
        frontier: PriorityQueue = PriorityQueue()
        start = Node(location=self._start, parent=None, cost=0, heuristic=self._heuristic(self._start))
        frontier.push(start)
        explored: Dict[Location, int] = {self._start: 0}
        while not frontier.empty:
            current_node: Node = frontier.pop()
            current_location: Location = current_node.location
            # update the screen
            self._maze.mark(location=current_location, color=Cell.EXPLORE)
            self._maze.draw()
            pygame.display.update()
            if self._goal_check(current_location):
                return current_node
            for child in self._successor(current_location, self._allow_diagonal):
                new_cost = current_node.cost + 1
                if child not in explored or explored[child] > new_cost:
                    frontier.push(Node(location=child, parent=current_node,
                                     cost=new_cost, heuristic=self._heuristic(child)))
                    explored[child] = new_cost
        return None

    @property
    def time(self):
        return self._time

    @property
    def len_of_path(self):
        return self._len_of_path


Finally, we need to add SearchController that take input from the user, which is:
What heuristic to use.
Allow for diagonal movement.
Start the algorithm
